In [22]:
import pdfplumber

with pdfplumber.open("RelatorioFechamentoCaixaBobina.pdf") as pdf:
    page = pdf.pages[0]
    texto = page.extract_text()

In [23]:
texto

'PRODUTOS RAINHA DO SABOR MARAGOGI\nGROUP DAMASCENO LTDA\nCNPJ: 47133502000154 - IE: 240389077\n435, 435 - SAO BENTO\nMARAGOGI - AL\nFechamento de Caixa\nMovimento: 10/11/2025 a 10/11/2025\nUsuário: Todos\nDAV\nVendas com DAV ( Documento Auxiliar de Venda)\nDescrição Tipo Valor Total\nDINHEIRO DN 183,00\nCARTAO DE DEBITO CD 182,00\nPIX PX 328,00\n693,00\nLISTA DAS SANGRIAS\nVaor TotalTipo Histórico Valor\nDAV OVOS 24,00\nDAV VASSOURA 30,00\n54,00\nTOTALIZADORES\nDescrição Tipo Valor Total\nDINHEIRO DN 129,00\nCARTAO DE DEBITO CD 182,00\nPIX PX 328,00\n639,00'

In [24]:
import re

def parse_dav(texto):
    padrao = r"[A-Z ]+\s+([A-Z]{2})\s+(\d{1,3}(?:\.\d{3})*,\d{2})"
    return re.findall(padrao, texto)

def parse_sangrias(texto):
    padrao = r"DAV\s+([A-Z ]+)\s+(\d{1,3}(?:\.\d{3})*,\d{2})"
    return re.findall(padrao, texto)

def separar_secoes(texto):
    # Marcações claras no relatório
    secoes = {
        "dav": "",
        "sangrias": "",
        "totalizadores": ""
    }

    # Quebra em blocos
    linhas = texto.splitlines()

    secao_atual = None

    for linha in linhas:
        l = linha.strip().upper()

        if "DAV" in l and "VENDAS" in l:
            secao_atual = "dav"
            continue

        if "LISTA DAS SANGRIAS" in l:
            secao_atual = "sangrias"
            continue

        if "TOTALIZADORES" in l:
            secao_atual = "totalizadores"
            continue

        if secao_atual:
            secoes[secao_atual] += linha + "\n"

    return secoes

def parse_relatorio(texto):
    secoes = separar_secoes(texto)

    # DAV (primeiro bloco)
    dav = parse_dav(secoes["dav"])

    # Sangrias
    sangrias = parse_sangrias(secoes["sangrias"])

    # Totalizadores (parte final)
    totalizadores = parse_dav(secoes["totalizadores"])

    return {
        "dav": dav,
        "sangrias": sangrias,
        "totalizadores": totalizadores
    }


In [25]:
import openpyxl
from openpyxl.styles import PatternFill
relatorio = openpyxl.load_workbook("CAIXA_format.xlsx", data_only=False)
caixa = relatorio['Planilha1']

dav = dict(parse_relatorio(texto)['dav'])
sangria = dict(parse_relatorio(texto)['sangrias'])

In [26]:
sangria



{'OVOS': '24,00', 'VASSOURA': '30,00'}

In [27]:
for ab, val in dav.items():
    if ab == 'DN':
        caixa['B60'] = val
    elif ab == 'CD':
        caixa['B61'] = val
    elif ab == 'PX':
        caixa['B62'] = val
    elif ab == 'PZ':
        caixa['B63'] = val
    

In [28]:
linha_init = 51
for i, (desc, valor) in enumerate(sangria.items()):
    linha = linha_init + i
    if linha > 56:
        break  # evita ultrapassar as 6 linhas da planilha

    caixa[f"A{linha}"] = desc.strip()
    caixa[f"B{linha}"] = valor


In [29]:
print("Gerando CAIXA fechado!")
try:
    relatorio.save('CAIXA_fechado.xlsx')
except:
    print("não foi possivel gerar o arquivo!\n")
else:
    print("arquivo gerado com sucesso!!!\n")

input("Pressione ENTER para fechar!!!!!!\n")

Gerando CAIXA fechado!
arquivo gerado com sucesso!!!



''